In [2]:
# importamos librerías
import pandas as pd
import numpy as np
import requests
import os
#from dotenv import load_dotenv
import camelot
from datetime import datetime
# Cargamos variables de entorno
#load_dotenv()

In [3]:
# endpoint de la API de datos de la Comunidad de Madrid
#url = os.getenv("API_URL")
url_histo = "https://datos.comunidad.madrid/api/3/action/package_show?id=calidad_aire_datos_historico"

In [4]:
# Hacemos la petición y comprobamos el estado
response_histo = requests.get(url_histo)
response_histo.status_code

200

In [5]:
# Convertimos a json
data_histo = response_histo.json()
data_histo.keys()

dict_keys(['help', 'success', 'result'])

In [6]:
# Comprobamos que hemos obtenido un diccionario de diccionarios
type(data_histo["result"])

dict

In [7]:
# seleccionamos el elemento del diccionario obtenido que contiene los resultados
resultado_histo = data_histo.get("result")
resultado_histo.keys()

dict_keys(['license_title', 'maintainer', 'relationships_as_object', 'private', 'maintainer_email', 'num_tags', 'id', 'metadata_created', 'owner_org', 'metadata_modified', 'author', 'author_email', 'state', 'version', 'license_id', 'type', 'resources', 'num_resources', 'tags', 'groups', 'creator_user_id', 'relationships_as_subject', 'name', 'isopen', 'url', 'notes', 'title', 'extras', 'license_url', 'organization'])

In [8]:
# seleccionamos el elemento del diccionario obtenido que contiene la información de los recursos disponibles
todos_annos = resultado_histo.get("resources")
type(todos_annos)

list

In [9]:
type(todos_annos[0])


dict

In [10]:
todos_annos[0].keys()

dict_keys(['cache_last_updated', 'cache_url', 'mimetype_inner', 'hash', 'description', 'metadata_modified', 'format', 'url', 'created', 'state', 'package_id', 'last_modified', 'mimetype', 'url_type', 'position', 'size', 'datastore_active', 'id', 'resource_type', 'name'])

In [11]:
for anno in todos_annos:
    print(anno["name"])

Descripción datos de contaminantes
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025


In [42]:
# periodo es una lista de diccionarios. Tendremos que recorrerla para obtener los enlaces a los ficheros
archivos = ["2024","Descripción datos de contaminantes"]
for archivo in archivos:
    for anno in todos_annos:
        if anno["name"] == archivo:
            datos_anno = anno.get("url")
            formato = anno.get("format")
            response_anno = requests.get(datos_anno)
            # Verificar si la descarga fue exitosa
            if response_anno.status_code == 200:
             # Guardar el archivo en tu computadora
                with open(f'../datos/madrid_{archivo}.{formato}', 'wb') as file:
                    file.write(response_anno.content)
                print("Archivo guardado exitosamente.")
            else:
                print(f"No se pudo descargar el archivo. Código de estado: {response_anno.status_code}")

Archivo guardado exitosamente.
Archivo guardado exitosamente.


In [43]:
# Ruta del archivo PDF
archivo_pdf = "../datos/madrid_Descripción datos de contaminantes.PDF"

# Extraer tablas
tablas = camelot.read_pdf(archivo_pdf, pages="3")

# Verificar si se extrajeron tablas
if len(tablas) > 0:
     tablas[0].to_csv(f"../datos/datos_contaminantes.csv")  # Guardar como CSV

else:
    print("No se encontraron tablas en la página indicada.")

datos_contaminantes = pd.read_csv("../datos/datos_contaminantes.csv", index_col=0)
datos_contaminantes

CropBox missing from /Page, defaulting to MediaBox


,DESCRIPCIÓN MAGNITUD,CÓDIGO \nTÉCNICA \nDE MEDIDA,DESCRIPCIÓN TÉCNICA \nDE MEDIDA,UNIDAD,DESCRIPCIÓN UNIDAD
CÓDIGO MAGNITUD,,,,,
1,Dióxido de azufre,38,Fluorescencia ultravioleta,µg/m³,microgramos por metro cúbico
6,Monóxido de carbono,48,Espectrometría infrarroja no \ndispersiva,mg/m³,miligramos por metro cúbico
7,Monóxido de nitrógeno,8,Quimioluminiscencia,µg/m³,microgramos por metro cúbico
8,Dióxido de nitrógeno,8,Quimioluminiscencia,µg/m³,microgramos por metro cúbico
9,"Partículas en suspensión < PM2,5",49,Absorción beta,µg/m³,microgramos por metro cubico
10,Partículas en suspensión < PM10,49,Absorción beta,µg/m³,microgramos por metro cubico
11,Partículas en suspensión < PM1,49,Absorción beta,µg/m³,microgramos por metro cubico
12,Óxidos de nitrógeno,8,Quimioluminiscencia,µg/m³,microgramos por metro cúbico
14,Ozono,6,Absorción ultravioleta,µg/m³,microgramos por metro cubico


In [44]:
madrid_2024 = pd.read_csv("../datos/madrid_2024.CSV", sep=";", index_col=0)
madrid_2024.head()

,municipio,estacion,magnitud,punto_muestreo,ano,mes,dia,h01,v01,h02,...,h20,v20,h21,v21,h22,v22,h23,v23,h24,v24
provincia,,,,,,,,,,,,,,,,,,,,,
28,102,1,1,28102001_1_38,2024,1,1,4,V,4,...,4,V,4,V,4,V,4,V,4,V
28,102,1,6,28102001_6_48,2024,1,1,"0,2",V,"0,2",...,"0,2",V,"0,1",V,"0,1",V,"0,1",V,"0,1",V
28,102,1,7,28102001_7_8,2024,1,1,2,V,2,...,1,V,1,V,2,V,1,V,1,V
28,102,1,8,28102001_8_8,2024,1,1,8,V,8,...,9,V,10,V,9,V,8,V,7,V
28,102,1,10,28102001_10_49,2024,1,1,5,V,6,...,8,V,8,V,9,V,8,V,8,V


In [48]:
# endpoint de la API de datos de la Comunidad de Madrid
#url = os.getenv("API_URL")
url_dia = "https://datos.comunidad.madrid/api/3/action/package_show?id=calidad_aire_datos_dia"
# Hacemos la petición y comprobamos el estado
response_dia = requests.get(url_dia)
response_dia.status_code

200

In [50]:
# Convertimos a json
dia = response_dia.json()
dia.keys()

dict_keys(['help', 'success', 'result'])

In [51]:
# seleccionamos el elemento del diccionario obtenido que contiene los resultados
resultado_dia = dia.get("result")
resultado_dia.keys()

dict_keys(['license_title', 'maintainer', 'relationships_as_object', 'private', 'maintainer_email', 'num_tags', 'id', 'metadata_created', 'owner_org', 'metadata_modified', 'author', 'author_email', 'state', 'version', 'license_id', 'type', 'resources', 'num_resources', 'tags', 'groups', 'creator_user_id', 'relationships_as_subject', 'name', 'isopen', 'url', 'notes', 'title', 'extras', 'license_url', 'organization'])

In [52]:
# seleccionamos el elemento del diccionario obtenido que contiene la información de los recursos disponibles
periodo_dia = resultado_dia.get("resources")
len(periodo_dia)

3

In [53]:
# es una lista de diccionarios, vamos a ver qué claves tiene el primer elemento
periodo_dia[0].keys()

dict_keys(['cache_last_updated', 'cache_url', 'mimetype_inner', 'hash', 'description', 'metadata_modified', 'format', 'url', 'created', 'state', 'package_id', 'last_modified', 'mimetype', 'url_type', 'position', 'size', 'datastore_active', 'id', 'resource_type', 'name'])

In [54]:
# podemos recorrer la lista buscando el elemento que tenga el formato csv y obtener la url
for elemento in periodo_dia:
    if elemento["format"] == "CSV":
        datos_dia = elemento.get("url")
        print(datos_dia)
        response = requests.get(datos_dia)
        hoy =  datetime.now().date()  
        # Verificar si la descarga fue exitosa
        if response.status_code == 200:
            # Guardar el archivo en tu computadora
            with open(f'../datos/madrid_{hoy}.csv', 'wb') as file:
                file.write(response.content)
                print("Archivo guardado exitosamente.")
        else:
                print(f"No se pudo descargar el archivo. Código de estado: {response.status_code}")

https://datos.comunidad.madrid/dataset/3dacd589-ecca-485c-81b9-a61606b7199f/resource/ee75d31c-8042-4067-bbd7-739bb3eb8059/download/calidad_aire_datos_dia.csv
Archivo guardado exitosamente.


In [55]:
# endpoint de la API de datos de la Comunidad de Madrid
#url = os.getenv("API_URL")
url_mes = "https://datos.comunidad.madrid/api/3/action/package_show?id=calidad_aire_datos_mes"
# Hacemos la petición y comprobamos el estado
response_mes = requests.get(url_mes)
response_mes.status_code

200

In [56]:
# Convertimos a json
mes = response_mes.json()
mes.keys()

dict_keys(['help', 'success', 'result'])

In [57]:
# seleccionamos el elemento del diccionario obtenido que contiene los resultados
resultado_mes = mes.get("result")
resultado_mes.keys()

dict_keys(['license_title', 'maintainer', 'relationships_as_object', 'private', 'maintainer_email', 'num_tags', 'id', 'metadata_created', 'owner_org', 'metadata_modified', 'author', 'author_email', 'state', 'version', 'license_id', 'type', 'resources', 'num_resources', 'tags', 'groups', 'creator_user_id', 'relationships_as_subject', 'name', 'isopen', 'url', 'notes', 'title', 'extras', 'license_url', 'organization'])

In [58]:
# seleccionamos el elemento del diccionario obtenido que contiene la información de los recursos disponibles
periodo_mes = resultado_mes.get("resources")
len(periodo_mes)

3

In [61]:
# podemos recorrer la lista buscando el elemento que tenga el formato csv y obtener la url
for elemento in periodo_mes:
    if elemento["format"] == "CSV":
        datos_mes = elemento.get("url")
        print(datos_mes)
        response = requests.get(datos_mes)
        hoy =  datetime.now().date()  
        # Verificar si la descarga fue exitosa
        if response.status_code == 200:
            # Guardar el archivo en tu computadora
            with open(f'../datos/madrid_{hoy.month}_{hoy.year}.csv', 'wb') as file:
                file.write(response.content)
                print("Archivo guardado exitosamente.")
        else:
                print(f"No se pudo descargar el archivo. Código de estado: {response.status_code}")

https://datos.comunidad.madrid/dataset/f5f78c68-ffa9-443b-905e-bcab7036bb95/resource/7aca4163-667b-48d5-8abf-7cc164a193be/download/calidad_aire_datos_mes.csv
Archivo guardado exitosamente.


In [24]:
type(periodo_mes[0].get("metadata_modified"))


str

In [25]:
https://datos.comunidad.madrid/dataset/f5f78c68-ffa9-443b-905e-bcab7036bb95/resource/7aca4163-667b-48d5-8abf-7cc164a193be/download/calidad_aire_datos_mes.csv

SyntaxError: invalid decimal literal (1184849039.py, line 1)